In [ ]:
pip install cmake

In [ ]:
pip install --upgrade pip setuptools wheel

In [ ]:
pip install dlib

In [ ]:
pip install face_recognition

In [ ]:
pip install scipy

In [ ]:
pip install threading

In [ ]:
pip install playsound

In [ ]:
pip install opencv-python opencv-python-headless

In [ ]:
pip install --upgrade opencv-python

In [1]:
import face_recognition
import cv2
import time
from scipy.spatial import distance as dist
import playsound
import threading
import numpy as np

In [2]:
MIN_EAR = 0.30                  # Minimum Eye Aspect Ratio
EYE_AR_CONSEC_FRAMES = 10       # Minimum number of frames required to confirm Drowsiness

COUNTER = 0                     # To count the number of frames
ALARM_ON = False                # To switch on / off the alarm


In [3]:
import playsound

def sound_alarm(sound_file):
    playsound.playsound(sound_file)


In [4]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1],eye[5])   # First vertical distance of eye closure (p1,p5)
    B = dist.euclidean(eye[2],eye[5])   # Second vertical distance of eye closure (p2,p4)
    C = dist.euclidean(eye[0],eye[3])   # Horizontal distance of eye closure (p0,p3)

    ear = (A+B)/(2*C)                   # Eye Aspect Ratio
    return ear

In [5]:
def main():
    global COUNTER,ALARM_ON
    sound_file = r"C:\Users\Dell\Downloads\Thoongathe_ringtone.mp3"
    
    video_capture = cv2.VideoCapture(0)
    video_capture.set(3,320)       #To resize the video capturing frame width  3-width to 320 pixels
    video_capture.set(4,240)       #To resize the video capturing frame height  4-height to 240 pixels
    while(True):
        ret,frame = video_capture.read()
        face_landmarks_list = face_recognition.face_landmarks(frame)
        for face_landmark in face_landmarks_list:
            leftEye = face_landmark['left_eye']
            rightEye = face_landmark['right_eye']

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR)/2

            lpts = np.array(leftEye)
            rpts = np.array(rightEye)

            cv2.polylines(frame,[lpts],True,(255,255,0),1)
            cv2.polylines(frame,[rpts],True,(255,255,0),1)

            if ear < MIN_EAR:
                COUNTER += 1
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    if not ALARM_ON:
                        ALARM_ON = True
                        t = threading.Thread(target=sound_alarm ,args=(sound_file,))
                        t.start()                    
                    cv2.putText(frame,"Alert You are feeling sleepy",(5,10),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,255),1)
                    
            else:
                COUNTER = 0
                ALARM_ON = False
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (200, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
        cv2.imshow("sleep detection",frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()
            

In [6]:
if __name__ == '__main__':
    main()